In [12]:
import pyspark as ps
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import udf, array
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
import boto
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )



In [13]:
sc = spark.sparkContext

In [14]:
def flow_casting_function(row):
    (time, duration,src_comp,src_port,dst_comp,dst_port,protocol,pk_count,byte_count) = row
    if(time=='?'):
        time = 666999666
    return (int(time),int(duration),src_comp,src_port,dst_comp,dst_port,protocol,int(pk_count),int(byte_count))



In [15]:

def auth_casting_function(row):
    (time, scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success) = row
    if(time=='?'):
        time = 666999666
    return (int(time),scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success)

In [16]:
from pyspark.sql.types import *

flow_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('duration',     LongType(), True),
    StructField('src_comp',   StringType(),  True),
    StructField('src_port',   StringType(),  True),
    StructField('dst_comp',   StringType(),  True),
    StructField('dst_port',   StringType(),  True),
    StructField('protocol',  StringType(), True),
    StructField('pk_count',  LongType(),  True),
    StructField('byte_count',  LongType(),  True)] )

In [17]:
auth_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('src_user_domain',   StringType(),  True),
    StructField('dst_user_domain',   StringType(),  True),
    StructField('src_comp',   StringType(),  True),
    StructField('dst_comp',   StringType(),  True),
    StructField('auth_type',  StringType(), True),
    StructField('logon_type',  StringType(),  True),
    StructField('auth_orientation',  StringType(),  True),
    StructField('Success',  StringType(),  True)] )

In [20]:
rdd_auth = (sc.textFile("Raws/auth.txt.gz")
            .map(lambda rowstr : rowstr.split(","))
             .map(auth_casting_function))

In [ ]:
auth_df = spark.createDataFrame(rdd_auth,auth_schema)

# create List of active Users

In [ ]:
add_success_value = udf(lambda x : 1 if x == 'Success' else 0 )
auth_df = auth_df.withColumn('Success_Value', add_success_value(auth_df['Success']))

In [ ]:
src_df = auth_df.filter("logon_type = 'Interactive'").groupBy('src_user_domain').agg({'Success_Value': 'mean','Success':'count'}).collect()



In [ ]:
src_df = pd.DataFrame(src_df)
src_df.columns = ['user','avg_success','count_success']
src_df = src_df.sort_values('count_success',ascending=False)

In [ ]:
src_DOM_df = src_df[[('DOM' in x and '$' not in x) for x in src_df['user']]]
active_users = list(src_DOM_df['user'])
active_auth_df = auth_df.filter("logon_type = 'Interactive'").where(col("src_user_domain").isin(active_users))

In [ ]:
active_comps_df = pd.DataFrame(active_auth_df.groupBy('src_comp').agg(countDistinct('src_user_domain')).collect())

In [ ]:
active_comps_df.to_csv('active_comps_df.csv')

In [ ]:
# Finished Saving Localy

In [4]:
rdd_flow = (sc.textFile('s3://galvanize-dsi/NetworkLogs/flows.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(flow_casting_function)
           )
flow_df = spark.createDataFrame(rdd_flow,flow_schema)



In [5]:
active_comps_df = pd.read_csv('active_comps_df.csv')
active_red_team =pd.read_csv('activered_team.csv')
active_comps_df.columns = ['ignore','comp','count_user']
reds = list(active_red_team['dst_comp'])

In [6]:
flow_df = spark.createDataFrame(rdd_flow,flow_schema)

In [7]:
active_flow_df = flow_df.where(col("src_comp").isin(list(active_comps_df['comp'])) | col("dst_comp").isin(list(active_comps_df['comp'])))
red_flow_df =  flow_df.where(col("src_comp").isin(reds) | col("dst_comp").isin(reds))



In [8]:
add_download = udf(lambda x : 1 if x in list(active_comps_df['comp']) else 0 )
add_upload = udf(lambda x : 1 if x in list(active_comps_df['comp']) else 0 )

add_red_download = udf(lambda x : 1 if x in list(reds) else 0 )
add_red_upload = udf(lambda x : 1 if x in list(reds) else 0 )


active_flow_df = active_flow_df.withColumn('Download', add_download(active_flow_df['dst_comp']))
active_flow_df = active_flow_df.withColumn('Upload', add_upload(active_flow_df['src_comp']))

flow_df = flow_df.withColumn('Download', add_download(flow_df['dst_comp']))
flow_df = flow_df.withColumn('Upload', add_upload(flow_df['src_comp']))

red_flow_df = red_flow_df.withColumn('Download', add_red_download(flow_df['dst_comp']))
red_flow_df = red_flow_df.withColumn('Upload', add_red_upload(flow_df['src_comp']))

In [9]:


red_flow_df = red_flow_df.filter(col('Upload').rlike("0")|col('Download').rlike("0"))
flow_df = flow_df.filter(col('Upload').rlike("0")|col('Download').rlike("0"))
active_flow_df = active_flow_df.filter(col('Upload').rlike("0")|col('Download').rlike("0"))

#active_flow_df = active_flow_df.filter(inter_check(array('Download','Upload')))
#flow_df = flow_df.filter(inter_check(array('Download','Upload')))

In [10]:
add_count = udf(lambda x : x[1] if x[0]=='1' else 0 )

In [11]:
red_flow_df = red_flow_df.withColumn('upload_bytes',add_count(array('Upload','byte_count')))
red_flow_df = red_flow_df.withColumn('download_bytes',add_count(array('Download','byte_count')))
red_flow_df = red_flow_df.withColumn('upload_pk',add_count(array('Upload','pk_count')))
red_flow_df = red_flow_df.withColumn('download_pk',add_count(array('Download','pk_count')))


In [12]:
active_flow_df = active_flow_df.withColumn('upload_bytes',add_count(array('Upload','byte_count')))
active_flow_df = active_flow_df.withColumn('download_bytes',add_count(array('Download','byte_count')))
active_flow_df = active_flow_df.withColumn('upload_pk',add_count(array('Upload','pk_count')))
active_flow_df = active_flow_df.withColumn('download_pk',add_count(array('Download','pk_count')))


In [13]:
flow_df = flow_df.withColumn('upload_bytes',add_count(array('Upload','byte_count')))
flow_df = flow_df.withColumn('download_bytes',add_count(array('Download','byte_count')))
flow_df = flow_df.withColumn('upload_pk',add_count(array('Upload','pk_count')))
flow_df = flow_df.withColumn('download_pk',add_count(array('Download','pk_count')))

# Add User / Service Columns

In [14]:
add_items = udf(lambda x : x[1] if x[0]=='1' else x[2] )

In [15]:
red_flow_df = red_flow_df.withColumn('user',add_count(array('Upload','src_comp','dst_comp')))
red_flow_df = red_flow_df.withColumn('service',add_count(array('Download','src_comp','dst_comp')))


In [16]:
flow_df = flow_df.withColumn('user',add_count(array('Upload','src_comp','dst_comp')))
flow_df = flow_df.withColumn('service',add_count(array('Download','src_comp','dst_comp')))


In [17]:
active_flow_df = active_flow_df.withColumn('user',add_count(array('Upload','src_comp','dst_comp')))
active_flow_df = active_flow_df.withColumn('service',add_count(array('Download','src_comp','dst_comp')))


In [18]:
interval_size = 86400

In [19]:
make_interval = udf(lambda x :  (x//interval_size)*interval_size)

In [20]:

active_flow_df = active_flow_df.withColumn('interval', make_interval(active_flow_df['time']))
flow_df = flow_df.withColumn('interval', make_interval(flow_df['time']))
red_flow_df = red_flow_df.withColumn('interval', make_interval(red_flow_df['time']))

In [ ]:
#Agregate over interval and user

flow_time_series = active_flow_df.groupBy('interval','user').agg({'duration': 'mean','pk_count':'sum','byte_count':'sum','upload_bytes':'sum','download_bytes':'sum','upload_pk':'sum','download_pk':'sum'}).collect()


In [ ]:
flow_time_series_df = pd.DataFrame(flow_time_series)
flow_time_series_df.columns = ['time','user','mean_duration','pk_count','byte_count','upload_bytes','download_bytes','upload_pk','download_pk']

flow_time_series_df.to_csv('flow_time_series_df.csv')

In [ ]:
flow_time_series_df['time'] = flow_time_series_df['time'].astype('int64')
flow_time_series_df = flow_time_series_df.sort_values('time')


plt.plot(flow_time_series_df['time'],flow_time_series_df['upload_bytes'],label = 'Upload')
plt.plot(flow_time_series_df['time'],flow_time_series_df['download_bytes'],label = 'Download')
plt.legend()

plt.plot(flow_time_series_df['time'],flow_time_series_df['download_bytes']/flow_time_series_df['upload_bytes'])

In [ ]:
reds = list(active_red_team['dst_comp'])

In [ ]:
red_flow_df =  flow_df.where(col("src_comp").isin(reds) | col("dst_comp").isin(reds))
red_time_series = red_flow_df.groupBy('interval','user').agg({'duration': 'mean','pk_count':'sum','byte_count':'sum','upload_bytes':'sum','download_bytes':'sum','upload_pk':'sum','download_pk':'sum'}).collect()

red_time_series_df = pd.DataFrame(red_time_series)
red_time_series_df.columns = ['time','user','mean_duration','pk_count','byte_count','upload_bytes','download_bytes','upload_pk','download_pk']




red_time_series_df

In [ ]:
red_time_series_df.to_csv('red_time_series_df.csv')

In [ ]:

red_time_series_df['time'] = red_time_series_df['time'].astype('int64')
red_time_series_df = red_time_series_df.sort_values('time')

In [ ]:
plt.plot(red_time_series_df['time'],red_time_series_df['upload_bytes'],label = 'Upload')
plt.plot(red_time_series_df['time'],red_time_series_df['download_bytes'],label = 'Download')
plt.legend()

In [ ]:
plt.plot(flow_time_series_df['time'],flow_time_series_df['download_bytes']/flow_time_series_df['upload_bytes'])